In [2]:
!pip install pulp
from pulp import *

In [8]:
nodes = ['FIELD', 'PUMP1', 'PUMP2', 'PUMP3', 'PUMP4', 'REFINE']

capacity = {
    ('FIELD','PUMP1'): 6,
    ('FIELD','PUMP2'): 4,
    ('PUMP1','PUMP3'): 3,
    ('PUMP1','PUMP4'): 2,
    ('PUMP2','PUMP3'): 2,
    ('PUMP2','PUMP4'): 5,
    ('PUMP3','REFINE'): 6,
    ('PUMP4','REFINE'): 4,
    ('REFINE', 'FIELD'): 999
}

links = list(capacity.keys())
links

[('FIELD', 'PUMP1'),
 ('FIELD', 'PUMP2'),
 ('PUMP1', 'PUMP3'),
 ('PUMP1', 'PUMP4'),
 ('PUMP2', 'PUMP3'),
 ('PUMP2', 'PUMP4'),
 ('PUMP3', 'REFINE'),
 ('PUMP4', 'REFINE'),
 ('REFINE', 'FIELD')]

In [15]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Maximum Flow Problem", LpMaximize)

# A dictionary of the output variables
x = LpVariable.dicts("x",links,0)

# Create objective function
prob += x[('REFINE', 'FIELD')]

for n in nodes:
    prob += (lpSum([x[(i,j)] for (i,j) in links if j == n]) == lpSum([x[(i,j)] for (i,j) in links if i == n])), "Flow Conservation in Node:%s"%n
    
for l in links:
    prob += (x[l]<=capacity[l]),  "Capacity Limit:%s" % (str(l))

In [17]:
prob

Maximum Flow Problem:
MAXIMIZE
1*x_('REFINE',_'FIELD') + 0
SUBJECT TO
Flow_Conservation_in_Node:FIELD: - x_('FIELD',_'PUMP1') - x_('FIELD',_'PUMP2')
 + x_('REFINE',_'FIELD') = 0

Flow_Conservation_in_Node:PUMP1: x_('FIELD',_'PUMP1') - x_('PUMP1',_'PUMP3')
 - x_('PUMP1',_'PUMP4') = 0

Flow_Conservation_in_Node:PUMP2: x_('FIELD',_'PUMP2') - x_('PUMP2',_'PUMP3')
 - x_('PUMP2',_'PUMP4') = 0

Flow_Conservation_in_Node:PUMP3: x_('PUMP1',_'PUMP3') + x_('PUMP2',_'PUMP3')
 - x_('PUMP3',_'REFINE') = 0

Flow_Conservation_in_Node:PUMP4: x_('PUMP1',_'PUMP4') + x_('PUMP2',_'PUMP4')
 - x_('PUMP4',_'REFINE') = 0

Flow_Conservation_in_Node:REFINE: x_('PUMP3',_'REFINE')
 + x_('PUMP4',_'REFINE') - x_('REFINE',_'FIELD') = 0

Capacity_Limit:('FIELD',_'PUMP1'): x_('FIELD',_'PUMP1') <= 6

Capacity_Limit:('FIELD',_'PUMP2'): x_('FIELD',_'PUMP2') <= 4

Capacity_Limit:('PUMP1',_'PUMP3'): x_('PUMP1',_'PUMP3') <= 3

Capacity_Limit:('PUMP1',_'PUMP4'): x_('PUMP1',_'PUMP4') <= 2

Capacity_Limit:('PUMP2',_'PUMP3'): x_

In [18]:
# Optimize

prob.solve()

# Print the status of the solved LP
print("Status = %s" % LpStatus[prob.status])
print("Total Cost = %f" % value(prob.objective))

Status = Optimal
Total Cost = 9.000000


In [19]:
# Print the value of the variables at the optimum
for l in links:
    if(x[l].varValue > 0):
        print("%s: %f" % (x[l].name, x[l].varValue))

x_('FIELD',_'PUMP1'): 5.000000
x_('FIELD',_'PUMP2'): 4.000000
x_('PUMP1',_'PUMP3'): 3.000000
x_('PUMP1',_'PUMP4'): 2.000000
x_('PUMP2',_'PUMP3'): 2.000000
x_('PUMP2',_'PUMP4'): 2.000000
x_('PUMP3',_'REFINE'): 5.000000
x_('PUMP4',_'REFINE'): 4.000000
x_('REFINE',_'FIELD'): 9.000000
